<a href="https://colab.research.google.com/github/realgermosen/Chatbot_adventist/blob/main/bibleAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

# sonnets.txt
#!gdown --id 108jAePKK4R3BVYBbYJZ32JWUwxeMg20K


In [3]:

# Define path for file with sonnets
#SONNETS_FILE = './sonnets.txt'
from google.colab import files
uploaded = files.upload()

In [4]:
# Read the data
import re
#'bibleText'
with open('/content/bibleText.txt') as f:
    data = f.read()
    pattern = '\d{7},\d*,\d*,\d*,(\D*)\n'
    result = re.findall(pattern, data)

# Convert to lower case and save as a list
#corpus = data.lower().split("\n")

In [5]:
corpus = result
print(corpus[0])
print(f"There are {len(corpus)} lines of sonnets\n")
print(f"The first 5 lines look like this:\n")


for i in range(5):
  print(corpus[i])



In the beginning God created the heaven and the earth.
There are 31103 lines of sonnets

The first 5 lines look like this:

In the beginning God created the heaven and the earth.
"And the earth was without form, and void; and darkness was upon the face of the deep. And the Spirit of God moved upon the face of the waters."
"And God said, Let there be light: and there was light."
"And God saw the light, that it was good: and God divided the light from the darkness."
"And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day."


In [6]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

#tokenizer.texts_to_sequences(corpus[0])

#okenizer.texts_to_sequences([corpus[0]])

tokenizer.texts_to_sequences([corpus[0]])[0]


[6, 1, 677, 27, 1298, 1, 170, 2, 1, 110]

In [7]:

# GRADED FUNCTION: n_gram_seqs
def n_gram_seqs(corpus, tokenizer):

    input_sequences = []

    ### START CODE HERE
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i + 1]
            input_sequences.append(n_gram_sequence)
    ### END CODE HERE

    return input_sequences


In [8]:

# Test your function with one example
first_example_sequence = n_gram_seqs([corpus[0]], tokenizer)

print("n_gram sequences for first example look like this:\n")
first_example_sequence

# Test your function with a bigger corpus
next_3_examples_sequence = n_gram_seqs(corpus[1:4], tokenizer)

print("n_gram sequences for next 3 examples look like this:\n")
next_3_examples_sequence

# Apply the n_gram_seqs transformation to the whole corpus
input_sequences = n_gram_seqs(corpus, tokenizer)

# Save max length
max_sequence_len = max([len(x) for x in input_sequences])

print(f"n_grams of input_sequences have length: {len(input_sequences)}")
print(f"maximum length of sequences is: {max_sequence_len}")



n_gram sequences for first example look like this:

n_gram sequences for next 3 examples look like this:

n_grams of input_sequences have length: 758566
maximum length of sequences is: 90


In [9]:

# GRADED FUNCTION: pad_seqs
def pad_seqs(input_sequences, maxlen):

    ### START CODE HERE
    padded_sequences = np.array(pad_sequences(input_sequences, maxlen=maxlen, padding='pre'))

    return padded_sequences
    ### END CODE HERE


In [10]:

# Test your function with the n_grams_seq of the first example
first_padded_seq = pad_seqs(first_example_sequence, len(first_example_sequence))
first_padded_seq

# Test your function with the n_grams_seq of the next 3 examples
next_3_padded_seq = pad_seqs(next_3_examples_sequence, max([len(s) for s in next_3_examples_sequence]))
next_3_padded_seq

# Pad the whole corpus
input_sequences = pad_seqs(input_sequences, max_sequence_len)

print(f"padded corpus has shape: {input_sequences.shape}")



padded corpus has shape: (758566, 90)


In [11]:

# GRADED FUNCTION: features_and_labels
def features_and_labels(input_sequences, total_words):

    ### START CODE HERE
    features = input_sequences[:, :-1]
    labels = input_sequences[:, -1]
    one_hot_labels = to_categorical(labels, num_classes=total_words)
    ### END CODE HERE

    return features, one_hot_labels


In [ ]:

# Test your function with the padded n_grams_seq of the first example
first_features, first_labels = features_and_labels(first_padded_seq, total_words)

print(f"labels have shape: {first_labels.shape}")
print("\nfeatures look like this:\n")
first_features

# Split the whole corpus
features, labels = features_and_labels(input_sequences, total_words)

print(f"features have shape: {features.shape}")
print(f"labels have shape: {labels.shape}")



labels have shape: (9, 12878)

features look like this:



In [ ]:

# GRADED FUNCTION: create_model
def create_model(total_words, max_sequence_len):

    model = Sequential()
    ### START CODE HERE
    model.add(Embedding(total_words, 100, input_length=max_sequence_len - 1))
    model.add(Bidirectional(LSTM(150)))
    model.add(Dense(total_words, activation='softmax'))
    # Compile the model
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    ### END CODE HERE

    return model


In [ ]:

# Get the untrained model
model = create_model(total_words, max_sequence_len)

# Train the model
history = model.fit(features, labels, epochs=50, verbose=1)

# Take a look at the training curves of your model

acc = history.history['accuracy']
loss = history.history['loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training accuracy')
plt.title('Training accuracy')

plt.figure()

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.title('Training loss')
plt.legend()

plt.show()
"""
def download_history():
  import pickle
  from google.colab import files

  with open('history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

  files.download('history.pkl')

download_history()
"""
seed_text = "Please God, help me"
next_words = 100

for _ in range(next_words):
    # Convert the text into sequences
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    # Pad the sequences
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    # Get the probabilities of predicting a word
    predicted = model.predict(token_list, verbose=0)
    # Choose the next word based on the maximum probability
    predicted = np.argmax(predicted, axis=-1).item()
    # Get the actual word from the word index
    output_word = tokenizer.index_word[predicted]
    # Append to the current text
    seed_text += " " + output_word

print(seed_text)